In [26]:
import numpy as np
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

def tracking_function(tracker, frame_id, bboxes, scores, best_conf, num_prev_bbox=None):
    
    detects = []
    predictions = []
    
    if len(scores)>0:
        for i in range(len(bboxes)):
            # remember to check
            if scores[i] <= best_conf:
                continue
            box = bboxes[i]
            score = scores[i]
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            detects.append([x_min, y_min, x_min+bbox_width, y_min+bbox_height, score])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
#             print(predictions[:-1])
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    print("frame_id {}, number of notrack pred {}, number of tracked_objects {}, their frame_id {}, box x, y {}".format(frame_id, len(bboxes), len(tracked_objects),
                                                                                                           [o.last_detection.data[-1] for o in tracked_objects], 
                                                                                                            [o.estimate[0] for o in tracked_objects]))
    print("hit counter {}".format([o.hit_counter for o in tracked_objects ]) )
    to_add_preds = []
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))

        #exclude those in the edge
        if (x_min + bbox_width >= 1279) or (y_min + bbox_height) >= 719 or (x_min <= 1) or (y_min <= 1):
            continue
        score = tobj.last_detection.scores[0]            
        track_pred = np.array([[x_min, y_min, bbox_width, bbox_height]])
        #to_add_preds.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
            
        
#     if (num_prev_bbox is None or (num_prev_bbox - len(predictions) + 1) >= len(to_add_preds)):
#         predictions.extend(to_add_preds)
        
    return predictions, tracker

In [32]:
# 1280 , 720
bboxes = [[[1,1,1,1]] for _ in range(7)] + [[] for _ in range(7)]
confs = [[1] for i in range(7)] + [[] for _ in range(7)]
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=2,
)          
for f in range(len(bboxes)):
    p, tracker = tracking_function(tracker, f, bboxes[f], confs[f], 0, num_prev_bbox=None)
    print(p)

frame_id 0, number of notrack pred 1, number of tracked_objects 0, their frame_id [], box x, y []
hit counter []
['1.00 1 1 1 1']
frame_id 1, number of notrack pred 1, number of tracked_objects 0, their frame_id [], box x, y []
hit counter []
['1.00 1 1 1 1']
frame_id 2, number of notrack pred 1, number of tracked_objects 1, their frame_id [2], box x, y [array([1.5, 1.5])]
hit counter [6]
['1.00 1 1 1 1']
frame_id 3, number of notrack pred 1, number of tracked_objects 1, their frame_id [3], box x, y [array([1.5, 1.5])]
hit counter [7]
['1.00 1 1 1 1']
frame_id 4, number of notrack pred 1, number of tracked_objects 1, their frame_id [4], box x, y [array([1.5, 1.5])]
hit counter [6]
['1.00 1 1 1 1']
frame_id 5, number of notrack pred 1, number of tracked_objects 1, their frame_id [5], box x, y [array([1.5, 1.5])]
hit counter [7]
['1.00 1 1 1 1']
frame_id 6, number of notrack pred 1, number of tracked_objects 1, their frame_id [6], box x, y [array([1.5, 1.5])]
hit counter [6]
['1.00 1 1 1

In [29]:
bboxes[0]

[[1, 1, 1, 1]]

In [10]:
confs

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]